In [1]:
print(packages_import_statement)


import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

import re
import time 

from pprint import pprint




In [3]:
path = "./Data/Furman_Neighborhood_Indicators.xlsx"
data = pd.read_excel(path,sheet_name="Data")

columns_to_keep = pd.read_excel(path,sheet_name="Columns to Keep BN")
columns_to_keep.head(5)

,Variable,Description,Sources
0,afford_le030_rct,"The share of occupied, recently available rent...","IPUMS-USA, University of Minnesota, U.S. Depar..."
1,afford_le080_rct,"The share of occupied, recently available rent...","IPUMS-USA, University of Minnesota, U.S. Depar..."
2,afford_le120_rct,"The share of occupied, recently available rent...","IPUMS-USA, University of Minnesota, U.S. Depar..."
3,crime_all_rt,"The number of serious crimes, those classified...","New York City Police Department, U.S. Census, ..."
4,pfn_fam14condo,The total number of pre-foreclosure notices is...,New York State Department of Financial Service...


In [4]:
data_cds = data[data['Region Type']=='Community District'].copy()
cols_to_use = list()
cols_not_in_data = list()
for c in columns_to_keep['Variable'].unique():
    if c in data_cds.columns:
        cols_to_use.append(c)
    else:
        cols_not_in_data.append(c)

In [5]:
print(f"{len(cols_to_use)} cols in data, {len(cols_not_in_data)} cols not in data")

62 cols in data, 1 cols not in data


In [6]:
cols_to_use

['afford_le030_rct',
 'afford_le080_rct',
 'afford_le120_rct',
 'crime_all_rt',
 'pfn_fam14condo',
 'gross_rent_0_1beds',
 'gross_rent_2_3beds',
 'hh_alone_pct',
 'hh_inc_med_adj',
 'hh_u18_pct',
 'income_diversity_ratio',
 'lp_all',
 'med_r_1f',
 'med_r_4f',
 'med_r_cn',
 'med_r_ot',
 'nb_permit_res_units',
 'pct_prof_ela',
 'pct_prof_math',
 'pop_65p_pct',
 'pop_bornstate_pct',
 'pop_commute_carfree_pct',
 'pop_disabled_pct',
 'pop_discon_youth_pct',
 'pop_edu_collp_pct',
 'pop_edu_nohs_pct',
 'pop_foreign_pct',
 'pop_num',
 'pop_pov_65p_pct',
 'pop_pov_pct',
 'pop_pov_u18_pct',
 'pop_race_asian_pct',
 'pop_race_black_pct',
 'pop_race_div_idx',
 'pop_race_hisp_pct',
 'pop_race_white_pct',
 'population_density',
 'prox_park_pct',
 'prox_subway_pct',
 'rburden_mod_ami_81_120',
 'rburden_mod_ami_le80',
 'rburden_sev_ami_81_120',
 'rburden_sev_ami_le80',
 'rent_asking_med',
 'rent_burden_med',
 'rent_burden_mod_pct',
 'rent_burden_sev_pct',
 'rent_gross_med_adj',
 'rent_gross_recent_med_

In [7]:
cols = ["Region ID","Region Display","Region Name","Year"]
cols.extend(cols_to_use)
cols_with_data = data_cds[data_cds['Year']>=2018][cols].count()[data_cds[cols].count()>0].keys()
data_cds[data_cds['Year']>=2018][cols_with_data]

,Region ID,Region Display,Region Name,Year,crime_all_rt,pfn_fam14condo,lp_all,med_r_1f,med_r_4f,med_r_cn,...,prox_subway_pct,rent_asking_med,serious_viol_rate,shd_420c_props,shd_421a_props,shd_all_sub_props,shd_all_sub_units,shd_ph_props,shd_ph_units,priv_evic_filing_rt
1799,101,BX 01,Mott Haven/Melrose,2018,16.6,121.0,37.0,8000000.0,4250000.0,1900000.0,...,NaN,4000.0,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,101,BX 01,Mott Haven/Melrose,2019,17.6,134.0,NaN,NaN,NaN,NaN,...,NaN,NaN,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1801,101,BX 01,Mott Haven/Melrose,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,15.0,22.0,9027.0,0.0,0.0,NaN
1820,102,BX 02,Hunts Point/Longwood,2018,28.2,89.0,16.0,9842890.0,3935170.0,3225000.0,...,NaN,3660.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1821,102,BX 02,Hunts Point/Longwood,2019,26.7,87.0,NaN,NaN,NaN,NaN,...,NaN,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,502,SI 02,South Beach/Willowbrook,2018,5.3,1880.0,218.0,568000.0,356000.0,330000.0,...,NaN,1800.0,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2997,502,SI 02,South Beach/Willowbrook,2019,4.4,1577.0,NaN,NaN,NaN,NaN,...,NaN,NaN,12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,502,SI 02,South Beach/Willowbrook,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,21.0,3690.0,3.0,1413.0,NaN
3017,503,SI 03,Tottenville/Great Kills,2018,3.1,2570.0,371.0,545000.0,380000.0,352500.0,...,NaN,1840.0,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
### 2020 Data
cols_2020_data = data_cds[data_cds['Year']>=2020][cols].count()[data_cds[data_cds['Year']>=2020][cols].count()>0].keys()
data_cds[data_cds['Year']>=2020][cols_2020_data]

,Region ID,Region Display,Region Name,Year,shd_420c_props,shd_421a_props,shd_all_sub_props,shd_all_sub_units,shd_ph_props,shd_ph_units
1801,101,BX 01,Mott Haven/Melrose,2020,0.0,15.0,22.0,9027.0,0.0,0.0
1822,102,BX 02,Hunts Point/Longwood,2020,0.0,9.0,15.0,1985.0,0.0,0.0
1843,103,BX 03,Morrisania/Crotona,2020,66.0,12.0,327.0,41586.0,36.0,14928.0
1864,104,BX 04,Highbridge/Concourse,2020,15.0,37.0,190.0,54196.0,11.0,1759.0
1885,105,BX 05,Fordham/University Heights,2020,2.0,8.0,38.0,9378.0,0.0,0.0
1906,106,BX 06,Belmont/East Tremont,2020,3.0,15.0,46.0,20107.0,3.0,492.0
1927,107,BX 07,Kingsbridge Heights/Bedford,2020,32.0,16.0,194.0,22722.0,22.0,5926.0
1948,108,BX 08,Riverdale/Fieldston,2020,6.0,5.0,56.0,15322.0,3.0,1323.0
1969,109,BX 09,Parkchester/Soundview,2020,68.0,4.0,312.0,15390.0,8.0,3478.0
1990,110,BX 10,Throgs Neck/Co-op City,2020,176.0,23.0,1039.0,42798.0,63.0,7766.0


In [9]:
### 2019 Data
cols_2019_data = data_cds[data_cds['Year']==2019][cols].count()[data_cds[data_cds['Year']==2019][cols].count()>0].keys()
data_cds[data_cds['Year']==2019][cols_2019_data]

,Region ID,Region Display,Region Name,Year,crime_all_rt,pfn_fam14condo,nb_permit_res_units,pct_prof_ela,pct_prof_math,serious_viol_rate
1800,101,BX 01,Mott Haven/Melrose,2019,17.6,134.0,209.0,0.857,0.847,4.9
1821,102,BX 02,Hunts Point/Longwood,2019,26.7,87.0,127.0,0.803,0.858,29.9
1842,103,BX 03,Morrisania/Crotona,2019,14.5,45.0,135.0,0.638,0.616,46.5
1863,104,BX 04,Highbridge/Concourse,2019,23.5,126.0,2650.0,0.686,0.620,25.0
1884,105,BX 05,Fordham/University Heights,2019,90.9,84.0,284.0,0.833,0.803,10.9
1905,106,BX 06,Belmont/East Tremont,2019,12.4,158.0,58.0,0.840,0.783,11.2
1926,107,BX 07,Kingsbridge Heights/Bedford,2019,8.5,168.0,322.0,0.788,0.757,41.0
1947,108,BX 08,Riverdale/Fieldston,2019,9.8,195.0,91.0,0.848,0.813,20.5
1968,109,BX 09,Parkchester/Soundview,2019,12.5,44.0,140.0,0.368,0.369,152.8
1989,110,BX 10,Throgs Neck/Co-op City,2019,16.6,219.0,467.0,0.286,0.228,122.5


In [10]:
### 2018 Data
cols_2018_data = data_cds[data_cds['Year']==2018][cols].count()[data_cds[data_cds['Year']==2018][cols].count()>0].keys()
data_cds[data_cds['Year']==2018][cols_2018_data]


,Region ID,Region Display,Region Name,Year,crime_all_rt,pfn_fam14condo,lp_all,med_r_1f,med_r_4f,med_r_cn,med_r_ot,nb_permit_res_units,pct_prof_ela,pct_prof_math,rent_asking_med,serious_viol_rate
1799,101,BX 01,Mott Haven/Melrose,2018,16.6,121.0,37.0,8000000.0,4250000.0,1900000.0,732390.0,113.0,0.858,0.832,4000.0,4.1
1820,102,BX 02,Hunts Point/Longwood,2018,28.2,89.0,16.0,9842890.0,3935170.0,3225000.0,722320.0,358.0,0.885,0.876,3660.0,28.0
1841,103,BX 03,Morrisania/Crotona,2018,14.2,56.0,13.0,5472400.0,1137500.0,1588470.0,612500.0,190.0,0.632,0.593,3150.0,42.7
1862,104,BX 04,Highbridge/Concourse,2018,24.4,126.0,25.0,5925000.0,1656250.0,1550000.0,398130.0,330.0,0.667,0.654,3470.0,20.0
1883,105,BX 05,Fordham/University Heights,2018,92.0,65.0,29.0,NaN,NaN,2036500.0,917400.0,186.0,0.758,0.758,4000.0,8.5
1904,106,BX 06,Belmont/East Tremont,2018,11.8,182.0,33.0,8450000.0,1950000.0,1205000.0,628970.0,405.0,0.806,0.744,3300.0,9.5
1925,107,BX 07,Kingsbridge Heights/Bedford,2018,9.0,168.0,32.0,11927500.0,1855830.0,1608890.0,526920.0,74.0,0.758,0.747,3200.0,25.5
1946,108,BX 08,Riverdale/Fieldston,2018,9.1,207.0,44.0,11150000.0,2245990.0,1650000.0,647060.0,593.0,0.844,0.798,2770.0,16.1
1967,109,BX 09,Parkchester/Soundview,2018,12.6,53.0,7.0,1662500.0,850000.0,596500.0,305840.0,258.0,0.336,0.311,2600.0,113.7
1988,110,BX 10,Throgs Neck/Co-op City,2018,17.0,262.0,43.0,2387500.0,700000.0,870000.0,340000.0,315.0,0.260,0.221,2430.0,80.3


In [26]:
cols_idx = ["Region Display",'Region ID',"Region Name","Year"]


#### 2020
"shd_ph_props"

#### 2019
"crime_all_rt"
                
#### 2018
["med_r_1f"
,"med_r_4f"
,"med_r_cn"
,"med_r_ot"
,"rent_asking_med"
]

data_2020 = \
data_cds[(data_cds['Year']==2020)][cols_idx+["shd_ph_props"]]
data_2020.columns = [f"{x}_20" if x not in cols_idx else x for x in data_2020.columns ]

data_2019 = \
data_cds[(data_cds['Year']==2019)][cols_idx+["crime_all_rt"]]
data_2019.columns = [f"{x}_19" if x not in cols_idx else x for x in data_2019.columns ]

data_2018 = \
data_cds[(data_cds['Year']==2018)][cols_idx+["med_r_1f"
,"med_r_4f"
,"med_r_cn"
,"med_r_ot"
,"rent_asking_med"]]
data_2018.columns = [f"{x}_18" if x not in cols_idx else x for x in data_2018.columns ]

print(len(data_2018),len(data_2019),len(data_2020))

data_to_save = data_2018.merge(right=data_2019,how='left',on=['Region Display','Region ID','Region Name']
                               ,suffixes=("_18","_19"))
data_to_save = data_to_save.merge(right=data_2020,how='left',on=['Region Display','Region ID','Region Name']
                               ,suffixes=("_","_20"))
data_to_save.drop(["Year_18","Year_19","Year"],axis=1).to_csv("../modeling_data/Xs__FurmanCenterCD_Data.csv")
data_to_save.drop(["Year_18","Year_19","Year"],axis=1)


59 59 58


,Region Display,Region ID,Region Name,med_r_1f_18,med_r_4f_18,med_r_cn_18,med_r_ot_18,rent_asking_med_18,crime_all_rt_19,shd_ph_props_20
0,BX 01,101,Mott Haven/Melrose,8000000.0,4250000.0,1900000.0,732390.0,4000.0,17.6,0.0
1,BX 02,102,Hunts Point/Longwood,9842890.0,3935170.0,3225000.0,722320.0,3660.0,26.7,0.0
2,BX 03,103,Morrisania/Crotona,5472400.0,1137500.0,1588470.0,612500.0,3150.0,14.5,36.0
3,BX 04,104,Highbridge/Concourse,5925000.0,1656250.0,1550000.0,398130.0,3470.0,23.5,11.0
4,BX 05,105,Fordham/University Heights,NaN,NaN,2036500.0,917400.0,4000.0,90.9,0.0
5,BX 06,106,Belmont/East Tremont,8450000.0,1950000.0,1205000.0,628970.0,3300.0,12.4,3.0
6,BX 07,107,Kingsbridge Heights/Bedford,11927500.0,1855830.0,1608890.0,526920.0,3200.0,8.5,22.0
7,BX 08,108,Riverdale/Fieldston,11150000.0,2245990.0,1650000.0,647060.0,2770.0,9.8,3.0
8,BX 09,109,Parkchester/Soundview,1662500.0,850000.0,596500.0,305840.0,2600.0,12.5,8.0
9,BX 10,110,Throgs Neck/Co-op City,2387500.0,700000.0,870000.0,340000.0,2430.0,16.6,63.0
